# Assignment 3: Texture synthesis
##### [Instruction](https://mattabrown.github.io/425/assignments/Assignment3.html)
##### Name: Yeongu Choe
##### Student number: 77672566

In [7]:
from PIL import Image, ImageDraw
import numpy as np
import random
import os.path
import pickle

### Question 4

In [8]:
def ComputeSSD(TODOPatch: np.ndarray, TODOMask: np.ndarray, textureIm: np.ndarray, patchL: int) -> np.ndarray:
    patch_rows, patch_cols, patch_bands = np.shape(TODOPatch)
    tex_rows, tex_cols, tex_bands = np.shape(textureIm)
    ssd_rows = tex_rows - 2 * patchL
    ssd_cols = tex_cols - 2 * patchL
    SSD = np.zeros((ssd_rows, ssd_cols))

    # Step1: Iterate over SSD
    for resultSSDRow in range(ssd_rows):
        for resultSSDColumn in range(ssd_cols):
            # Step2: Iterate over patch
            for patchRow in range(patch_rows):
                for patchColumn in range(patch_cols):
                    # Step3: Check if it has valid pixel value, looking at TODOMask
                    if (TODOMask[patchRow, patchColumn] == 0):
                        # Step4: Find Row and Column in terms of original image input
                        universalRowAddress = resultSSDRow+patchRow
                        universalColumnAddress = resultSSDColumn+patchColumn
                        # Step5: Convert data type into float32
                        currentPatchPixel = TODOPatch[patchRow][patchColumn].astype(
                            np.float32)
                        currentTextureImPixel = textureIm[universalRowAddress][universalColumnAddress].astype(
                            np.float32)
                        # Step6: Calculate Sum Squared Difference
                        difference = currentPatchPixel-currentTextureImPixel
                        differenceSquared = difference**2
                        # Step7: Assign summation of calculated SSD from above, into corresponding pixel in SSD matrix
                        SSD[resultSSDRow][resultSSDColumn] += np.sum(
                            differenceSquared)
    return SSD

### Question 5

In [9]:
def CopyPatch(imHole: np.ndarray, TODOMask: np.ndarray, textureIm: np.ndarray, iPatchCenter: np.int64, jPatchCenter: np.int64, iMatchCenter: np.int64, jMatchCenter: np.int64, patchL: int) -> np.ndarray:
    patchSize = 2 * patchL + 1

    for currentRowInPatch in range(patchSize):
        for currentColumnInPatch in range(patchSize):
            if (TODOMask[currentRowInPatch][currentColumnInPatch] == 1):
                # PatchCenter: center of the missing region on original image
                # Step1: Start from the top left corner of the missing region
                # Step1.1: Calculate row of the pixel that is going to be updated
                patchRow = (iPatchCenter-patchL)+currentRowInPatch
                # Step1.2: Calculate column of the pixel that is going to be updated
                patchColumn = (jPatchCenter-patchL)+currentColumnInPatch

                # MatchCenter: center of region that was selected, based on SSD samilarity
                # Step2: Start from the top left corner of the selected region
                # Step2.1: Calculate row of the pixel that is going to be updated
                textureRow = (iMatchCenter-patchL)+currentRowInPatch
                # Step2.2: Calculate column of the pixel that is going to be updated
                textureColumn = (jMatchCenter-patchL)+currentColumnInPatch

                # Step3: Update RGB pixel value for the missing region
                # Update Red channel
                imHole[patchRow][patchColumn][0] = textureIm[textureRow][textureColumn][0]
                # Update Green channel
                imHole[patchRow][patchColumn][1] = textureIm[textureRow][textureColumn][1]
                # Update Blue channel
                imHole[patchRow][patchColumn][2] = textureIm[textureRow][textureColumn][2]

    return imHole

#### Original Image
![Donkey original image](donkey.jpg)
#### Processed Image
![Donkey processed image](results.jpg)

### Question 6
#### Instruction
1. Change imname to new image file.
2. select area, running polyselect.py
3. set standard deviation (randomPatchSD) and patch length (patchL).

#### Well performed algorithm example
##### Original Image
![Original Image](Image/originalCloud.jpeg)
##### Processed Image
![Processed Image](Image/modifiedCloud.jpg)
* I tried to remove cloud from the original image.
* I used 0.3 for standard deviation and 3 for patch length.

#### Poorly performed algorithm example
##### Original Image
![Original Image](Image/originalSea.jpeg)
##### Processed Image
![Processed Image](Image/modifiedSea.jpg)
* I tried to remove the sun from the original image.
* I used 10 for standard deviation and 10 for patch length.

### Question 7

#### Effect of randomPatchSD
* The randomPatchSD is the standard deviation used to select a patch from the sample texture list.
* If the randomPatchSD value is too small, then only few types of patches will be used for filling the missing region. The processed image may contain artifact, because the function is choosing the patch from a small pool of patches.
* If randomPatchSD value is too large, then too many types of patchs can be selected to fill the missing region. Unrelated patchs may have put next to each other in the missing region.

#### Effect of patchL
* The patchL refers to the width and height of a square patch.
* If patchL is too small, the synthesized part in the processed image may not accurately capture the original pattern.
* If patchL is too large, various types of patches can be selected to fill the same missing region.